# Import libraries

In [1]:
import os
import sys
import subprocess
import tensorflow as tf

2025-06-13 06:23:13.362527: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749788593.379558 2535281 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749788593.384734 2535281 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749788593.398993 2535281 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749788593.399011 2535281 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749788593.399013 2535281 computation_placer.cc:177] computation placer alr

# Setup GPU

In [2]:
# List all available GPU devices
gpus = tf.config.experimental.list_physical_devices('GPU')

# If GPUs are available, enable dynamic memory allocation
if gpus:
    try:
        for gpu in gpus:
            # Prevent TensorFlow from allocating all GPU memory at once.
            # Instead, it will allocate memory as needed (useful for multi-tasking or sharing GPUs).
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # This exception occurs if set_memory_growth is called after the GPUs have been initialized
        print(e)

# Initialize a MirroredStrategy for distributed training across multiple GPUs
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


I0000 00:00:1749788595.844522 2535281 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21798 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


# Input Parameters

In [3]:
# Dataset parameters
source_dir = '/home/hmorales/WorkSpace/Development/ImageSegmentation/data/BC/'        # Path to dataset
psf_path = '/home/hmorales/WorkSpace/Development/ImageSegmentation/data/PSF.tif'
code_dir = '/home/hmorales/WorkSpace/Development/ImageSegmentation/code/'    # Path to code
out_dir = '/home/hmorales/WorkSpace/Development/ImageSegmentation/results/'
out_name = 'BC'

In [17]:
# Add code_dir to PYTHONPATH so that 'src' can be imported
env = os.environ.copy()
env["PYTHONPATH"] = code_dir + ":" + env.get("PYTHONPATH", "")

# Define paths
train_dir = os.path.join(source_dir, 'training_data')
test_dir  = os.path.join(source_dir, 'test_data')

patches_train_path    = os.path.join(train_dir, 'regular_patches')
patches_test_path     = os.path.join(test_dir, 'regular_patches')
patches_test_rec_path = os.path.join(test_dir, 'reconstruction_patches')

generate_patches_script_path = os.path.join(code_dir, 'src', 'generate_patches.py')
train_script_path            = os.path.join(code_dir, 'src', 'modeling', 'train.py')
predict_script_path          = os.path.join(code_dir, 'src', 'modeling', 'predict.py')
plots_script_path            = os.path.join(code_dir, 'src',  'plots.py')

models_path = os.path.join(code_dir, 'models')
out_images_path = os.path.join(out_dir, 'predicted_masks')
out_plots_path = os.path.join(out_dir, 'plots')


# Create datasets for train and test 

In [5]:
# Run training data patch generation without overlap
subprocess.run(["python", generate_patches_script_path, train_dir, 'False'], env=env)

2025-06-13 06:23:16.077 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: /home/hmorales/WorkSpace/Development/ImageSegmentation/code
2025-06-13 06:23:16.079 | INFO     | __main__:generate_patches:94 - Found 1 images to process
2025-06-13 06:23:16.079 | INFO     | __main__:generate_patches:95 - Using patch size: (64, 64, 64)
2025-06-13 06:23:16.079 | INFO     | __main__:generate_patches:96 - Using step size: 64 (without overlap)
Saving patches for img3:   0%|          | 0/60 [00:00<?, ?it/s]
                                                               2025-06-13 06:23:16.195 | INFO     | __main__:generate_patches:177 - Generated 60 patches of size (64, 64, 64) for img3.tif
Processing images: 100%|██████████| 1/1 [00:00<00:00,  8.68it/s]
2025-06-13 06:23:16.195 | SUCCESS  | __main__:generate_patches:185 - Patch generation complete!


CompletedProcess(args=['python', '/home/hmorales/WorkSpace/Development/ImageSegmentation/code/src/generate_patches.py', '/home/hmorales/WorkSpace/Development/ImageSegmentation/data/BC/training_data', 'False'], returncode=0)

In [6]:
# Run test data patch generation with overlap (for reconstruction)
subprocess.run(["python", generate_patches_script_path, test_dir, 'True'], env=env)

# Run test data patch generation without overlap (for evaluation)
subprocess.run(["python", generate_patches_script_path, test_dir, 'False'], env=env)

2025-06-13 06:23:16.436 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: /home/hmorales/WorkSpace/Development/ImageSegmentation/code
2025-06-13 06:23:16.437 | INFO     | __main__:generate_patches:94 - Found 2 images to process
2025-06-13 06:23:16.437 | INFO     | __main__:generate_patches:95 - Using patch size: (64, 64, 64)
2025-06-13 06:23:16.438 | INFO     | __main__:generate_patches:96 - Using step size: 56 (with overlap for reconstruction)
Processing images:   0%|          | 0/2 [00:00<?, ?it/s]2025-06-13 06:23:16.613 | INFO     | __main__:generate_patches:126 - Applying padding for reconstruction to img1.tif: ((16, 16), (8, 8), (8, 8))

Saving patches for img1: 100%|█████████▉| 1194/1196 [00:01<00:00, 849.64it/s]
                                                                             2025-06-13 06:23:18.446 | INFO     | __main__:generate_patches:177 - Generated 1196 patches of size (64, 64, 64) for img1.tif
Processing images:  50%|█████     | 1/2 [00:02<00:02,  2.01s/

CompletedProcess(args=['python', '/home/hmorales/WorkSpace/Development/ImageSegmentation/code/src/generate_patches.py', '/home/hmorales/WorkSpace/Development/ImageSegmentation/data/BC/test_data', 'False'], returncode=0)

# Train models

In [7]:
# Basic training without augmentation
subprocess.run([
    "python", 
    train_script_path, 
    "UNet3D", 
    patches_train_path, 
    out_name,
    "--psf",
    psf_path,
    "--augmentation", 
    "NONE"
], env=env)



2025-06-13 06:23:23.991561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749788604.008131 2535953 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749788604.013388 2535953 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749788604.026835 2535953 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749788604.026851 2535953 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749788604.026852 2535953 computation_placer.cc:177] computation placer alr

Epoch 1/100


I0000 00:00:1749788610.907560 2536257 cuda_dnn.cc:529] Loaded cuDNN version 90300


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 679ms/step - accuracy: 0.6741 - loss: 0.6929

6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.6959 - loss: 0.6928 - val_accuracy: 0.9663 - val_loss: 0.6915
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 679ms/step - accuracy: 0.9683 - loss: 0.6909

...

6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9920 - loss: 0.0192 - val_accuracy: 0.9888 - val_loss: 0.0272
Epoch 72/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 774ms/step - accuracy: 0.9922 - loss: 0.0188 - val_accuracy: 0.9886 - val_loss: 0.0280
Epoch 73/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 770ms/step - accuracy: 0.9921 - loss: 0.0190 - val_accuracy: 0.9888 - val_loss: 0.0275
Epoch 74/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 779ms/step - accuracy: 0.9924 - loss: 0.0182 - val_accuracy: 0.9887 - val_loss: 0.0273
Epoch 75/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 699ms/step - accuracy: 0.9924 - loss: 0.0182

6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9924 - loss: 0.0182 - val_accuracy: 0.9888 - val_loss: 0.0270
Epoch 76/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 764ms/step - accuracy: 0.9923 - loss: 0.0185 - val_accuracy: 0.9889 - val_loss: 0.0271
Epoch 77/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 686ms/step - accuracy: 0.9925 - loss: 0.0182

6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9925 - loss: 0.0182 - val_accuracy: 0.9892 - val_loss: 0.0259
Epoch 78/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 761ms/step - accuracy: 0.9922 - loss: 0.0187 - val_accuracy: 0.9891 - val_loss: 0.0261
Epoch 79/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 763ms/step - accuracy: 0.9928 - loss: 0.0173 - val_accuracy: 0.9890 - val_loss: 0.0270
Epoch 80/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 767ms/step - accuracy: 0.9927 - loss: 0.0175 - val_accuracy: 0.9890 - val_loss: 0.0262
Epoch 81/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 766ms/step - accuracy: 0.9929 - loss: 0.0171 - val_accuracy: 0.9890 - val_loss: 0.0274
Epoch 82/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 764ms/step - accuracy: 0.9929 - loss: 0.0169 - val_accuracy: 0.9884 - val_loss: 0.0274


2025-06-13 06:32:16.294 | SUCCESS  | __main__:main:241 - Training complete!


CompletedProcess(args=['python', '/home/hmorales/WorkSpace/Development/ImageSegmentation/code/src/modeling/train.py', 'UNet3D', '/home/hmorales/WorkSpace/Development/ImageSegmentation/data/BC/training_data/regular_patches', 'BC', '--psf', '/home/hmorales/WorkSpace/Development/ImageSegmentation/data/PSF.tif', '--augmentation', 'NONE'], returncode=0)

In [8]:
# Training with standard augmentation
subprocess.run([
    "python", 
    train_script_path, 
    "UNet3D", 
    patches_train_path, 
    out_name, 
    "--psf",
    psf_path,    
    "--augmentation", 
    "STANDARD"
], env=env)

# Training with advanced augmentation 
subprocess.run([
    "python", 
    train_script_path, 
    "UNet3D",
    patches_train_path,    
    out_name, 
    "--psf",
    psf_path,      
    "--augmentation", 
    "OURS"
], env=env)

2025-06-13 06:32:18.173063: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749789138.189840 2566062 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749789138.194852 2566062 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749789138.208279 2566062 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749789138.208297 2566062 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749789138.208299 2566062 computation_placer.cc:177] computation placer alr

Epoch 1/100


I0000 00:00:1749789145.295374 2566377 cuda_dnn.cc:529] Loaded cuDNN version 90300


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 686ms/step - accuracy: 0.6753 - loss: 0.6929

6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.6969 - loss: 0.6928 - val_accuracy: 0.9663 - val_loss: 0.6915
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 688ms/step - accuracy: 0.9683 - loss: 0.6909

...

6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.9726 - loss: 0.0468 - val_accuracy: 0.9666 - val_loss: 0.0492
Epoch 98/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.9717 - loss: 0.0459 - val_accuracy: 0.9666 - val_loss: 0.0501
Epoch 99/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9739 - loss: 0.0419

6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.9736 - loss: 0.0422 - val_accuracy: 0.9665 - val_loss: 0.0491
Epoch 100/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9718 - loss: 0.0456

6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.9719 - loss: 0.0456 - val_accuracy: 0.9667 - val_loss: 0.0488


2025-06-13 06:59:37.799 | SUCCESS  | __main__:main:241 - Training complete!


CompletedProcess(args=['python', '/home/hmorales/WorkSpace/Development/ImageSegmentation/code/src/modeling/train.py', 'UNet3D', '/home/hmorales/WorkSpace/Development/ImageSegmentation/data/BC/training_data/regular_patches', 'BC', '--psf', '/home/hmorales/WorkSpace/Development/ImageSegmentation/data/PSF.tif', '--augmentation', 'OURS'], returncode=0)

# Generate Predictions

In [12]:
subprocess.run([
    "python", 
    predict_script_path, 
    patches_test_rec_path,
    test_dir,
    out_name, 
    models_path, 
    "--output-dir",
    out_images_path,
], env=env)


2025-06-13 07:13:00.230701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749791580.247456 2653031 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749791580.252515 2653031 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749791580.266181 2653031 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749791580.266198 2653031 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749791580.266200 2653031 computation_placer.cc:177] computation placer alr

CompletedProcess(args=['python', '/home/hmorales/WorkSpace/Development/ImageSegmentation/code/src/modeling/predict.py', '/home/hmorales/WorkSpace/Development/ImageSegmentation/data/BC/training_data/reconstruction_patches', '/home/hmorales/WorkSpace/Development/ImageSegmentation/data/BC/test_data', 'BC', '/home/hmorales/WorkSpace/Development/ImageSegmentation/code/models', '--output-dir', '/home/hmorales/WorkSpace/Development/ImageSegmentation/results/predicted_masks'], returncode=1)

# Generate Plots

In [18]:
subprocess.run([
    "python", 
    plots_script_path, 
    patches_test_rec_path,
    patches_test_path,
    test_dir,
    out_name, 
    models_path, 
    "--output-dir",
    out_plots_path,
], env=env)




2025-06-13 07:54:55.513 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: /home/hmorales/WorkSpace/Development/ImageSegmentation/code
2025-06-13 07:54:56.233414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749794096.250560 2690548 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749794096.255758 2690548 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749794096.269728 2690548 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749794096.269747 2690548 computation_placer.cc:177] computation placer already registered. Please check link

CompletedProcess(args=['python', '/home/hmorales/WorkSpace/Development/ImageSegmentation/code/src/plots.py', '/home/hmorales/WorkSpace/Development/ImageSegmentation/data/BC/test_data/reconstruction_patches', '/home/hmorales/WorkSpace/Development/ImageSegmentation/data/BC/test_data/regular_patches', '/home/hmorales/WorkSpace/Development/ImageSegmentation/data/BC/test_data', 'BC', '/home/hmorales/WorkSpace/Development/ImageSegmentation/code/models', '--output-dir', '/home/hmorales/WorkSpace/Development/ImageSegmentation/results/plots'], returncode=0)

In [ ]:
print("All calculations are successfully finished")